In [2]:
import numpy as np

Estimar pi 

In [25]:
N = 10000000
#para que los resultados sean reproducibles
np.random.seed(42)
dardo = np.random.random([N, 2])

dentro = np.zeros(N)
for i in range(N):
    if dardo[i, 0] ** 2 + dardo[i, 1] ** 2 < 1:
        dentro[i] = 1

todo = np.array([1] * N)

pi = np.sum(dentro)/np.sum(todo)*4
print('pi:', pi)

pi: 3.1415988


In [28]:
4*len(np.where(np.sqrt(dardo[:,0]**2 + dardo[:,1]**2) < 1)[0])/len(dardo)

3.1415988

In [ ]:
# fijar la semilla, para reproducir los cálulos
np.random.seed(42)

# Datos sintéticos, con ruido gaussiano
# La idea de esto es que podrían ser nuestras mediciones.
true_slope = 2.5
true_intercept = 1.
x_data = np.linspace(0, 1, 1_000)
sigma_guass_noise = .5

y_data = true_slope * x_data + true_intercept + np.random.normal(0, sigma_guass_noise, len(x_data))

# Log-likelihood
def log_likelihood(theta, x, y, sigma):
    a, b = theta
    model = a * x + b
    output = -.5 * np.sum(((y-model) / sigma) ** 2)
    return output

# Metropolis-Hasstings algorithm
def metropolis_hastings(x, y, sigma, n_chain, initial_guess, proposal_sigma):
    # Primero calculamos la likelihood actual
    n_dim = len(initial_guess) # Cuantos parametos buscamos
    chain = np.zeros((n_chain, n_dim)) # Aqui guardamos los parametros
    chain[0] = initial_guess
    current_likelihood = log_likelihood(chain[0], x, y, sigma)

    # Ahora queremos comparar para ver si actualizamos el estado.
    # Para ello, necesitamos calcular la nueva likelihood, generar un
    # numero aleatorio y comparar.
    for idx in range(1, n_chain):
        # Nuevos parametros
        proposal = chain[idx - 1] + np.random.normal(0, proposal_sigma, n_dim)
        proposal_likelihood = log_likelihood(proposal, x, y, sigma)

        # Probabilidad de aceptar 
        alpha = np.exp(proposal_likelihood - current_likelihood)

        # Generamos numero aleatorio y comparamos.
        if np.random.random() < alpha:
            chain[idx] = proposal
            current_likelihood = proposal_likelihood
        else:
            chain[idx] = chain[idx-1]
    
    return chain

In [ ]:
initial_guess = [1.,1.]
n_chain = 10_000
proposal_sigma = .1

chain = metropolis_hastings(x_data, y_data, sigma_guass_noise, n_chain, initial_guess, proposal_sigma)

# Descartamos el "burn-in"
# chain = chain[100:]x

# definimos chi cuadrado
def chi_square(x, y, a, b, sigma):
    model_prediction = a * x + b
    output = np.sum(((model_prediction - y)/sigma)**2)
    return output

chi_square_values = np.array([chi_square(x_data, y_data, a, b, sigma_guass_noise) for a, b in chain])